In [6]:
# imports
import os, re, json, logging
from typing import List, Dict
import fitz          
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

#find table

In [5]:
# ---------- Page discovery utilities ----------
MIN_CODES = 5                                           # tweak if a report is sparse
CODE_RE   = re.compile(r'\b(?:BP|GOV|SBM|IRO|E[1-5]|S[1-4]|G[1-3])[-–‐]?\d{1,2}\b', re.I)

def code_cnt(txt: str) -> int:
    "Count ESRS-style code tokens in a block of text."
    return len(CODE_RE.findall(txt))

def find_esrs_pages(pdf_path: str) -> List[int]:
    """
    Return a **1-based** list of pages that most likely hold the ESRS
    cross-reference table, using the heuristic you provided.
    """
    doc = fitz.open(pdf_path)

    # 1️⃣ “ESRS table of contents” segment
    toc_hits = [
        i for i, p in enumerate(doc)
        if "esrs table of contents" in p.get_text().lower()
        and code_cnt(p.get_text()) >= MIN_CODES
    ]
    if toc_hits:
        start  = toc_hits[-1]
        pages  = [start]
        for j in range(start + 1, len(doc)):
            if code_cnt(doc[j].get_text()) >= MIN_CODES:
                pages.append(j)
            else:
                break
        return [p + 1 for p in pages]                     # 1-based return

    # 2️⃣ densest contiguous block anywhere in the PDF
    qualifies = [code_cnt(p.get_text()) >= MIN_CODES for p in doc]
    blocks, i = [], 0
    while i < len(doc):
        if not qualifies[i]:
            i += 1; continue
        start = i
        while i < len(doc) and qualifies[i]:
            i += 1
        blocks.append(range(start, i))
    if not blocks:
        return []

    best = max(blocks, key=lambda r: len(r)*sum(code_cnt(doc[p].get_text()) for p in r))
    return [p + 1 for p in best]                          # 1-based list


In [11]:
import fitz

try:
    # Update path to local Reports directory
    doc = fitz.open("Reports/PhilipsFullAnnualReport2024-English.pdf")
    text = doc[242].get_text()  # zero-indexed
    print(text[:1000])  # show first 1000 characters
finally:
    # Ensure document is closed
    if 'doc' in locals():
        doc.close()

8.7 
ESRS cross-reference table
ESRS 2 
General disclosures
Disclosure Requirement BP-1 – General basis for preparation of sustainability
Sustainability statement: General basis for preparation
169 Limited Assurance
Disclosure Requirement BP-2 – Disclosures in relation to specific circumstances
Sustainability statement: General basis for preparation
169 Limited Assurance
Disclosure Requirement GOV-1 – The role of the administrative, management and 
supervisory bodies
SFDR/BRR
Environmental, Social and Governance: ESG governance
Supervisory Board report: composition, diversity and self-evaluation
Other Board-related matters: Diversity
40
64
268
Limited Assurance
Disclosure Requirement GOV-2 – Information provided to and sustainability matters 
addressed by the undertaking’s administrative, management and supervisory bodies
Environmental, Social and Governance: ESG governance
40 Limited Assurance
Disclosure Requirement GOV-3 - Integration of sustainability-related performance in incentiv

In [1]:
pdf_path = "Reports/PhilipsFullAnnualReport2024-English.pdf"
pages = find_esrs_pages(pdf_path)
print("ESRS pages:", pages)

NameError: name 'find_esrs_pages' is not defined

#LLM APY

In [5]:
def clean_page_refs(ref_str: str, current_page: int) -> str:
    """
    Remove the footer page number and stray single-digit footnotes.
    Return a comma-separated string of real page refs (or "").
    """
    if not ref_str:
        return ""

    toks  = re.split(r"[,\s]+", ref_str.strip())
    refs  = []
    for t in toks:
        if t.isdigit():
            n = int(t)
            if n == current_page:          # footer → drop
                continue
            if n < 10:                     # footnote marker → drop
                continue
            refs.append(str(n))
    return ", ".join(refs)

In [6]:

# ---------- LLM extraction ----------
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# constants for the final table
def get_company_name(pdf_path: str) -> str:
    """Extract company name from PDF filename more robustly"""
    filename = os.path.basename(pdf_path).lower()
    
    # Remove common words and file extensions
    clean_name = (filename
                 .replace('-', ' ')
                 .replace('_', ' ')
                 .replace('.pdf', '')
                 .replace('annual', '')
                 .replace('report', '')
                 .replace('group', '')
                 .replace('integrated', '')
                 .strip())
    
    # Take first word as company name if it's meaningful
    first_word = clean_name.split()[0].title()
    
    # Validate if first_word is a reasonable company name (>2 chars)
    if len(first_word) > 2:
        return first_word
    
    # Fallback: Return filename without common suffixes
    return os.path.splitext(os.path.basename(pdf_path))[0]


CATEGORY       = "DR"
MAX_PAGE_COLS  = 6           # Page_ref1 … Page_ref5
COMPANY = get_company_name(pdf_path)


def extract_page_items(text: str, page_num: int, client: OpenAI) -> pd.DataFrame:
    """
    Call GPT once for a single page and return a **ready-made DataFrame**
    with columns:  name, category, variable, value,
                   Page_ref1 … Page_ref{MAX_PAGE_COLS}
    """
    prompt = f"""You are an ESRS disclosure analyzer. Extract **all** ESRS disclosures
from the text below (page {page_num}). 
* The large single number printed as the page footer (e.g. **{page_num}**)
  is **NOT** a page reference.  Ignore it completely.
* If the “Pages” column for a disclosure is blank, return an empty
  page_reference "".
* value = 1 if a page is given, 0 if not
Return JSON exactly like:
{{"items":[{{"code":"","title":"","page_reference":""}}]}}  — no extra keys.

Text:
{text}"""

    try:
        resp  = client.chat.completions.create(
            model="gpt-4o",   # or "gpt-4o" / "gpt-4.1"
            messages=[{"role":"user","content":prompt}],
            response_format={"type":"json_object"},
        )
        data = json.loads(resp.choices[0].message.content)
        items: List[Dict] = data.get("items", [])

    except Exception as e:
        logging.error(f"LLM error on page {page_num}: {e}")
        items = []

    # ---------- reshape to “ideal” layout ----------
    if not items:         # nothing found on that page
        return pd.DataFrame(columns=["name","category","variable","value"]
                                      + [f"Page_ref{i}" for i in range(1, MAX_PAGE_COLS+1)])

    df = pd.DataFrame(items)

    # 1) make page_reference always a string
    df["page_reference"] = (
        df["page_reference"]
        .apply(lambda x: ", ".join(map(str, x))          # list → "40, 64, 268"
                            if isinstance(x, (list, tuple)) else str(x or ""))
    )

    # 2) now clean + derive 'value' exactly as before
    df["page_reference"] = df["page_reference"].apply(
        lambda s: clean_page_refs(s, page_num)
    )
    # COLLAPSE rows with the same code  (NEW)
    df = (df
        .groupby("code", as_index=False, sort=False)          # keep original order
        .agg({"page_reference": ", ".join}))                  # "40, 64, 268"


    df["value"] = (df["page_reference"].str.len() > 0).astype(int)

    df = (df
          .assign(
              name     = COMPANY,
              category = CATEGORY,
              variable = df.get("code"),
          )
          .drop(columns=["title"], errors="ignore")                       # keep if you want it
    )

    pages = (df["page_reference"]
         .str.replace(r"\s+", " ", regex=True).str.strip()        # tidy spaces/line-breaks
         .str.split(r"[,\s]+", expand=True)                       # ← use regex
         .rename(columns=lambda i: f"Page_ref{i+1}"))


    # ensure we always have Page_ref1 … Page_ref{MAX_PAGE_COLS}
    for i in range(pages.shape[1]+1, MAX_PAGE_COLS+1):
        pages[f"Page_ref{i}"] = ""

    df = pd.concat([df.drop(columns=["page_reference"]), pages], axis=1)

    # final column order
    df = df[["name","category","variable","value"]
            + [f"Page_ref{i}" for i in range(1, MAX_PAGE_COLS+1)]]

    return df.fillna("")        # blanks instead of NaN


In [66]:
pdf_path = "Reports/PhilipsFullAnnualReport2024-English.pdf"

pages = find_esrs_pages(pdf_path)
print("Chosen pages:", pages)

doc = fitz.open(pdf_path)

dfs = []                                            # << DataFrames live here
for p in pages:
    text = doc[p-1].get_text()          # PyMuPDF uses 0-based index
    df_page = extract_page_items(text, p, client)
    dfs.append(df_page)                 # << append, do NOT extend

full_table = pd.concat(dfs, ignore_index=True)
display(full_table.style.hide(axis="index"))
# full_table.to_excel("Philips_ESRS.xlsx", index=False)

Chosen pages: [243, 244, 245, 246, 247]


name,category,variable,value,Page_ref1,Page_ref2,Page_ref3,Page_ref4,Page_ref5,Page_ref6
Philipsfull2024,DR,BP-1,1,169,,,,,
Philipsfull2024,DR,BP-2,1,169,,,,,
Philipsfull2024,DR,GOV-1,1,40,64,268,,,
Philipsfull2024,DR,GOV-2,1,40,,,,,
Philipsfull2024,DR,GOV-3,1,76,80,,,,
Philipsfull2024,DR,GOV-4,1,170,,,,,
Philipsfull2024,DR,GOV-5,1,40,58,,,,
Philipsfull2024,DR,SBM-1,1,54,170,,,,
Philipsfull2024,DR,SBM-2,1,54,170,,,,
Philipsfull2024,DR,SBM-3,1,54,170,174,,,


In [33]:
def process_reports_folder():
    reports_dir = "Reports"
    pdf_files = [f for f in os.listdir(reports_dir) if f.endswith('.pdf')]
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(reports_dir, pdf_file)
        print(f"\nProcessing: {pdf_file}")
        try:
            pages = find_esrs_pages(pdf_path)
            print(f"ESRS pages in {pdf_file}:", pages)
        except Exception as e:
            print(f"Error processing {pdf_file}: {e}")

# Run folder processing
process_reports_folder()


Processing: allianz-group-annual-report-2024.pdf
ESRS pages in allianz-group-annual-report-2024.pdf: [157, 158, 159, 160]

Processing: adidas-ar24.pdf
ESRS pages in adidas-ar24.pdf: [143, 144, 145]

Processing: entire-full-report-basf-ar24.pdf
ESRS pages in entire-full-report-basf-ar24.pdf: [147, 148, 149, 150]

Processing: sopra_steria_urd_2024_en_opti.pdf
ESRS pages in sopra_steria_urd_2024_en_opti.pdf: [233, 234, 235, 236, 237]

Processing: Randstad_Annual_Report_2024_0.pdf
ESRS pages in Randstad_Annual_Report_2024_0.pdf: [141, 142, 143]

Processing: PhilipsFullAnnualReport2024-English.pdf
ESRS pages in PhilipsFullAnnualReport2024-English.pdf: [243, 244, 245, 246, 247]

Processing: Annual Report 2024_Zalando SE_EN_250503_s.pdf
ESRS pages in Annual Report 2024_Zalando SE_EN_250503_s.pdf: [282, 283, 284, 285, 286, 287, 288]

Processing: sap-2024-integrated-report.pdf
ESRS pages in sap-2024-integrated-report.pdf: [222, 223, 224, 225, 226, 227]

Processing: Volkswagen_2024_e.pdf
ESRS p